In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = mls
band = [9]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [0]
  ng_refs = [7]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(2, 1.6, 1.6, 1.6, 1.6, 1.6, 1.8)]
  wgt = [(0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.003586                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.000077
0.000750      2.0 -0.003578                  2.0 -0.000087
0.001052      3.0 -0.003559                  3.0 -0.000029
0.001476      4.0 -0.003536                  4.0  0.000004
0.002070      5.0 -0.003511                  5.0  0.000036
0.002904      6.0 -0.003485                  6.0  0.000068
0.004074      7.0 -0.003457                  7.0  0.000101
0.005714      8.0 -0.003431                  8.0  0.000136
0.008015      9.0 -0.003403                  9.0  0.000172
0.011243     10.0 -0.003376                 10.0  0.000209
0.015771     11.0 -0.003350                 11.0  0.000248
0.022122     12.0 -0.003329                 12.0  0.000288
0.031031     13.0 -0.003315                 13.0  0.000329
0.043528     14.0 -0.003309                 14.0  0.000371
0.061057     15.0 -0.003206                 15.0  0.000512
0.085645     16.0 -0.002705                 16.0  0.000984
0.120136     17.0 -0.001532                 17.0  0.001905
0.168516     18.0  0.000364                 18.0  0.003249
0.236378     19.0  0.003208                 19.0  0.005145
0.331549     20.0  0.007301                 20.0  0.007774
0.465100     21.0  0.013032                 21.0  0.011374
0.652400     22.0  0.020888                 22.0  0.016280
0.915100     23.0  0.028761                 23.0  0.021259
1.283650     24.0  0.028229                 24.0  0.021716
1.800600     25.0  0.020295                 25.0  0.017792
2.525700     26.0  0.013084                 26.0  0.013578
3.542800     27.0  0.008076                 27.0  0.010082
4.969550     28.0  0.004895                 28.0  0.007365
6.970850     29.0  0.003042                 29.0  0.005355
9.778100     30.0  0.001969                 30.0  0.003857
13.715850    31.0  0.001262                 31.0  0.002679
19.239350    32.0  0.000763                 32.0  0.001758
26.987250    33.0  0.000492                 33.0  0.001132
37.855300    34.0  0.000360                 34.0  0.000720
53.100050    35.0  0.000204                 35.0  0.000370
73.887500    36.0  0.000017                 36.0  0.000060
97.662500    37.0 -0.000096                 37.0 -0.000130
121.437500   38.0 -0.000149                 38.0 -0.000227
145.212500   39.0 -0.000234                 39.0 -0.000320
168.987500   40.0 -0.000288                 40.0 -0.000371
192.762500   41.0 -0.000166                 41.0 -0.000299
216.537500   42.0  0.000813                 42.0  0.000480
240.312500   43.0  0.003293                 43.0  0.002605
264.087500   44.0  0.007068                 44.0  0.006263
287.862500   45.0  0.011686                 45.0  0.011789
311.637500   46.0  0.015158                 46.0  0.014901
335.412500   47.0  0.019409                 47.0  0.017875
359.187500   48.0  0.025189                 48.0  0.023304
382.962500   49.0  0.032281                 49.0  0.030908
406.737500   50.0  0.040512                 50.0  0.039318
430.512500   51.0  0.049589                 51.0  0.047602
454.287500   52.0  0.058272                 52.0  0.054810
478.062500   53.0  0.067152                 53.0  0.062682
501.837500   54.0  0.076627                 54.0  0.072269
525.612500   55.0  0.086571                 55.0  0.083636
549.387500   56.0  0.096936                 56.0  0.095877
573.162500   57.0  0.107898                 57.0  0.108174
596.937500   58.0  0.123729                 58.0  0.124229
620.712500   59.0  0.138122                 59.0  0.137489
644.487500   60.0  0.149943                 60.0  0.147382
668.262500   61.0  0.159523                 61.0  0.155137
692.037500   62.0  0.167059                 62.0  0.162226
715.812500   63.0  0.173049                 63.0  0.169907
739.587500   64.0  0.177992                 64.0  0.178618
76

# Fluxes by Layer

CRD                              CLIRAD  \
                         flug          fldg      fnetg       flug   
pressure     level                                                  
0.000000e+00 1     -16.108743  0.000000e+00 -16.108743        NaN   
1.000000e-08 1            NaN           NaN        NaN -16.411749   
6.244000e-04 2     -16.108744  8.405096e-08 -16.108744 -16.411749   
8.759000e-04 3     -16.108744  1.184119e-07 -16.108744 -16.411749   
1.228600e-03 4     -16.108744  1.681485e-07 -16.108744 -16.411749   
1.723400e-03 5     -16.108744  2.402944e-07 -16.108744 -16.411749   
2.417400e-03 6     -16.108745  3.449184e-07 -16.108744 -16.411749   
3.390900e-03 7     -16.108745  4.966543e-07 -16.108745 -16.411749   
4.756500e-03 8     -16.108746  7.166952e-07 -16.108745 -16.411749   
6.672000e-03 9     -16.108747  1.035446e-06 -16.108746 -16.411750   
9.358900e-03 10    -16.108749  1.496943e-06 -16.108747 -16.411750   
1.312780e-02 11    -16.108751  2.164873e-06 -16.108749 -16.411750   
1.841450e-02 12    -16.108754  3.130662e-06 -16.108751 -16.411750   
2.583020e-02 13    -16.108758  4.524967e-06 -16.108754 -16.411751   
3.623230e-02 14    -16.108764  6.535416e-06 -16.108758 -16.411752   
5.082340e-02 15    -16.108773  9.435278e-06 -16.108764 -16.411753   
7.129060e-02 16    -16.108785  1.390238e-05 -16.108771 -16.411754   
1.000000e-01 17    -16.108803  2.202975e-05 -16.108781 -16.411755   
1.402710e-01 18    -16.108826  3.835314e-05 -16.108788 -16.411755   
1.967600e-01 19    -16.108856  7.087237e-05 -16.108785 -16.411751   
2.759970e-01 20    -16.108890  1.345888e-04 -16.108755 -16.411737   
3.871000e-01 21    -16.108916  2.571929e-04 -16.108659 -16.411703   
5.431000e-01 22    -16.108907  4.888673e-04 -16.108418 -16.411623   
7.617000e-01 23    -16.108793  9.156824e-04 -16.107878 -16.411451   
1.068500e+00 24    -16.108462  1.629992e-03 -16.106832 -16.411124   
1.498800e+00 25    -16.107925  2.531439e-03 -16.105393 -16.410652   
2.102400e+00 26    -16.107328  3.385410e-03 -16.103942 -16.410120   
2.949000e+00 27    -16.106734  4.104154e-03 -16.102630 -16.409571   
4.136600e+00 28    -16.106169  4.674716e-03 -16.101494 -16.409027   
5.802500e+00 29    -16.105645  5.116897e-03 -16.100528 -16.408507   
8.139200e+00 30    -16.105164  5.477822e-03 -16.099686 -16.408025   
1.141700e+01 31    -16.104729  5.807610e-03 -16.098922 -16.407604   
1.601470e+01 32    -16.104375  6.140180e-03 -16.098234 -16.407288   
2.246400e+01 33    -16.104164  6.512633e-03 -16.097652 -16.407144   
3.151050e+01 34    -16.104143  7.018756e-03 -16.097124 -16.407225   
4.420010e+01 35    -16.104360  7.776293e-03 -16.096584 -16.407604   
6.200000e+01 36    -16.104997  8.842846e-03 -16.096155 -16.408469   
8.577500e+01 37    -16.106333  1.022688e-02 -16.096106 -16.410064   
1.095500e+02 38    -16.108049  1.167256e-02 -16.096376 -16.411990   
1.333250e+02 39    -16.110045  1.324828e-02 -16.096797 -16.414152   
1.571000e+02 40    -16.112353  1.489865e-02 -16.097454 -16.416553   
1.808750e+02 41    -16.115052  1.678758e-02 -16.098264 -16.419248   
2.046500e+02 42    -16.119238  2.050722e-02 -16.098730 -16.423313   
2.284250e+02 43    -16.129234  3.279367e-02 -16.096440 -16.433041   
2.522000e+02 44    -16.149114  6.194755e-02 -16.087167 -16.452383   
2.759750e+02 45    -16.184206  1.169452e-01 -16.067260 -16.486318   
2.997500e+02 46    -16.241225  2.068765e-01 -16.034349 -16.541234   
3.235250e+02 47    -16.316120  3.244613e-01 -15.991658 -16.613131   
3.473000e+02 48    -16.407203  4.702079e-01 -15.936995 -16.700418   
3.710750e+02 49    -16.515533  6.494763e-01 -15.866056 -16.804169   
3.948500e+02 50    -16.641891  8.667474e-01 -15.775144 -16.925077   
4.186250e+02 51    -16.786742  1.125691e+00 -15.661050 -17.063595   
4.424000e+02 52    -16.950511  1.429120e+00 -15.521391 -17.220171   
4.661750e+02 53    -17.130402  1.773123e+00 -15.357279 -17.392149   
4.899500e+02 54    -17.325032  2.156873e+00 -15.168159 -17.578077   
5.137250e+02 55    -17.5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')